
# Google Cluster Data — Colab EDA (Auto-Download)

This notebook is designed for **Google Colab**. It will:
- Create a workspace under `/content/google_cluster_data`
- **Download** the 2019 Google Cluster Data schema and a small sample of `task_usage`
- **Decompress** the sample
- Run first-pass EDA geared toward **cloud autoscaling**

> Tip: If you want files to persist, mount Google Drive in the next cell.


In [24]:
from google.colab import auth
auth.authenticate_user()


In [26]:
# Replace with your real project id if different
PROJECT_ID="rl-semester-project"

# Set the active project (prints the value)
gcloud config set project "$PROJECT_ID"

echo -e "\n== Active account =="
gcloud auth list --filter=status:ACTIVE --format="value(account)"

echo -e "\n== Active project =="
gcloud config get-value project


Are you sure you wish to set property [core/project] to rl-semester-project?

Do you want to continue (Y/n)?  

Command killed by keyboard interrupt

^C

== Active account ==
bm3pk@virginia.edu

== Active project ==
rl-semester-project


To switch the authenticated Google Cloud account, re-run the `auth.authenticate_user()` cell above. When the authentication pop-up appears, select or log in with your personal Google account (`bruce.mcgregor22@gmail.com`).

In [27]:
from google.colab import auth
auth.authenticate_user()

# Replace with your actual new Project ID
PROJECT_ID = "RL Semester Project"  # example — use your real project ID
print("Project ID:", PROJECT_ID)

import os, subprocess
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
subprocess.run(["gcloud", "config", "set", "project", PROJECT_ID], check=False)


Project ID: RL Semester Project


CompletedProcess(args=['gcloud', 'config', 'set', 'project', 'RL Semester Project'], returncode=1)

In [23]:
from google.colab import auth
auth.authenticate_user()

import os, subprocess
PROJECT_ID = "rl-semester-project"   # <-- keep this exact if that’s your project id
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
subprocess.run(["gcloud", "config", "set", "project", PROJECT_ID], check=False)

print("== Active account ==")
subprocess.run(["gcloud", "auth", "list"])

print("\n== Active project ==")
subprocess.run(["gcloud", "config", "get-value", "project"])


== Active account ==

== Active project ==


CompletedProcess(args=['gcloud', 'config', 'get-value', 'project'], returncode=0)

In [22]:
# 1) Authenticate
from google.colab import auth
auth.authenticate_user()

# 2) Set your exact Project ID (NOT the project *name*)
PROJECT_ID = "rl-semester-project"  # e.g., rl-semester-123456
assert "-" in PROJECT_ID, "This looks like a project *name*. Use the Project *ID*."

import os, subprocess
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
subprocess.run(["gcloud", "config", "set", "project", PROJECT_ID], check=False)

# 3) (Optional) List your projects to confirm the correct ID
subprocess.run(["gcloud", "projects", "list"])

# 4) Enable the BigQuery API for this project
subprocess.run(
    ["gcloud", "services", "enable", "bigquery.googleapis.com", f"--project={PROJECT_ID}"],
    check=False
)

# 5) Verify it's enabled
subprocess.run(["gcloud", "services", "list", "--enabled", f"--project={PROJECT_ID}"])


CompletedProcess(args=['gcloud', 'services', 'list', '--enabled', '--project=rl-semester-project'], returncode=1)

In [17]:
YOUR_EMAIL = "bruce.mcgregor22@gmail.com"

# Enable BigQuery API (safe to run again)
subprocess.run(["gcloud", "services", "enable", "bigquery.googleapis.com", f"--project={PROJECT_ID}"], check=False)

# Grant roles that allow running queries (creating jobs)
subprocess.run([
    "gcloud", "projects", "add-iam-policy-binding", PROJECT_ID,
    "--member", f"user:{YOUR_EMAIL}",
    "--role", "roles/bigquery.jobUser"
], check=False)

# 'bigquery.user' also works (includes job creation and listing)
subprocess.run([
    "gcloud", "projects", "add-iam-policy-binding", PROJECT_ID,
    "--member", f"user:{YOUR_EMAIL}",
    "--role", "roles/bigquery.user"
], check=False)

# (Optional) see your effective IAM on this project
subprocess.run(["gcloud", "projects", "get-iam-policy", PROJECT_ID, "--format=value(bindings)"])


CompletedProcess(args=['gcloud', 'projects', 'get-iam-policy', 'rl-semester-project', '--format=value(bindings)'], returncode=1)

In [21]:
!gcloud auth list


  Credentialed Accounts
ACTIVE  ACCOUNT
*       bm3pk@virginia.edu

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [20]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
print("Connected to:", client.project)

df = client.query("""
  SELECT name, SUM(number) AS total
  FROM `bigquery-public-data.usa_names.usa_1910_current`
  WHERE state='VA'
  GROUP BY name
  ORDER BY total DESC
  LIMIT 5
""").to_dataframe()
df


Connected to: rl-semester-project


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/rl-semester-project/jobs?prettyPrint=false: Access Denied: Project rl-semester-project: User does not have bigquery.jobs.create permission in project rl-semester-project.

Location: None
Job ID: 2317b316-cf95-482b-bed6-660a3c513eaf



## Basic Info & Nulls


In [ ]:

print("Columns:", list(df.columns))
print("\nShape:", df.shape)
print("\nNull counts (top 20):\n", df.isna().sum().sort_values(ascending=False).head(20))
df.describe(include='all').T.head(20)



## Numeric Columns (Common Autoscaling Signals)


In [ ]:

candidate_numeric = ['cpu_rate', 'mem_usage', 'disk_io_time', 'disk_space', 'assigned_memory', 'page_cache']
numeric_cols = [c for c in candidate_numeric if c in df.columns]
print("Numeric columns found:", numeric_cols)

if numeric_cols:
    display(df[numeric_cols].describe())
else:
    print("No expected numeric columns found; check schema and column names.")



## Correlations


In [ ]:

if len(numeric_cols) >= 2:
    corr = df[numeric_cols].corr()
    corr
else:
    print("Not enough numeric columns to compute correlations.")



## Distributions


In [ ]:

for col in numeric_cols:
    plt.figure()
    df[col].dropna().hist(bins=50)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()



## Time Conversion
Attempts to convert `start_time` / `end_time` to datetimes. The trace often uses **nanoseconds since epoch**.


In [ ]:

for tcol in ['start_time', 'end_time']:
    if tcol in df.columns:
        try:
            mean_val = df[tcol].dropna().astype(float).mean()
            if mean_val > 1e12:
                df[tcol] = pd.to_datetime(df[tcol], unit='ns', origin='unix', errors='coerce')
            else:
                df[tcol] = pd.to_datetime(df[tcol], unit='s', origin='unix', errors='coerce')
        except Exception as e:
            print(f"Warning: could not convert {tcol} -> datetime:", e)

[c for c in ['start_time','end_time'] if c in df.columns]



## Mean CPU by Hour


In [ ]:

if 'start_time' in df.columns and 'cpu_rate' in df.columns:
    ts = df[['start_time','cpu_rate']].dropna().set_index('start_time').sort_index()
    hourly = ts['cpu_rate'].resample('1H').mean()
    display(hourly.head())
    plt.figure()
    hourly.plot()
    plt.title("Mean CPU rate by hour")
    plt.xlabel("Time")
    plt.ylabel("Mean CPU rate")
    plt.show()
else:
    print("Missing 'start_time' and/or 'cpu_rate'.")



## Lag Features (Predictive Signals)
Downsample to 5-minute intervals, then create lagged features.


In [ ]:

if 'start_time' in df.columns and 'cpu_rate' in df.columns:
    ts5 = df[['start_time','cpu_rate']].dropna().set_index('start_time').sort_index()['cpu_rate'].resample('5T').mean()
    ts5 = ts5.interpolate(limit_direction='both')
    lagged = pd.DataFrame({
        'cpu_t': ts5,
        'cpu_t_1': ts5.shift(1),
        'cpu_t_2': ts5.shift(2),
        'cpu_t_12': ts5.shift(12),  # ~1 hour lag at 5-min res
    }).dropna()
    display(lagged.head())
    print("\nLag correlations:")
    display(lagged.corr())
else:
    print("Cannot compute lag features without 'start_time' and 'cpu_rate'.")



## Grouping (Machine/Job/Task)


In [ ]:

group_keys = [c for c in ['machine_id', 'job_id', 'task_index'] if c in df.columns]
if 'cpu_rate' in df.columns and group_keys:
    key = group_keys[0]
    g = df[[key, 'cpu_rate']].dropna().groupby(key)['cpu_rate'].mean().sort_values(ascending=False).head(10)
    print(f"Top 10 {key} by mean CPU_rate:")
    display(g)
else:
    print("No grouping keys found or 'cpu_rate' missing.")



## Save Small Parquet Sample (optional)
Saves a random subset for faster reloads. Will attempt to install `pyarrow` if needed.


In [ ]:

out_parquet = "/content/google_cluster_data/data_sample/sample_eda.parquet"
try:
    df.sample(min(len(df), 200_000), random_state=17).to_parquet(out_parquet, index=False)
    print(f"Saved sample to {out_parquet}")
except Exception as e:
    print("Parquet save failed; attempting to install pyarrow...")
    try:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pyarrow"])
        df.sample(min(len(df), 200_000), random_state=17).to_parquet(out_parquet, index=False)
        print(f"Saved sample to {out_parquet}")
    except Exception as e2:
        print("Parquet still unavailable. Skipping parquet export.", e2)



### Next Steps
- Concatenate additional `task_usage` parts for larger windows of time.
- Join with `task_events` / `machine_events` to add scheduling and capacity context.
- Engineer autoscaling features (rolling means/percentiles, sustained high-CPU windows).
- Prototype scaling policies: threshold-based vs. predictive vs. RL.
